# Curso de Python - Parte 3

## 16. Decoradores, introducción, para que sirven y cómo crear decoradores.

Un decorador en Python no es más que un nombre que se le da a un patrón de diseño. Los decoradores alteran de forma dinámica la funcionalidad de una función, método o clase, sin tener que usar subclases o cambiar el código fuente de la función que está siendo decorada.


De forma sencilla, un decorador no es más que un objeto invocable, una función o un objeto que implementa el método `__call__`, que recibe como otro objeto invocable.


In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("Antes de ejecutar la función")
        result = func(*args, **kwargs)
        print("Después de ejecutar la función")
        return result
    return wrapper


def dummy_function(value):
    return value


decorated_dummy_function = dec(dummy_function)
print(decorated_dummy_function(42))

Python incluye azúcar sintáctica para simplificar el uso de los decoradores, de forma que para decorar una función, basta con usar `@`. En el ejemplo anterior:

In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("Antes de ejecutar la función")
        result = func(*args, **kwargs)
        print("Después de ejecutar la función")
        return result
    return wrapper


@dec
def dummy_function(value):
    return value


print(dummy_function(42))

Un decorador también puede recibir parámetros, para lo que necesitamos añadir una nueva función de envoltura.

In [ ]:
def dec(param=False):
    def _dec(func):
        def wrapper(*args, **kwargs):
            if param:
                print("Se ha pasado True")
            else:
                print("Se ha pasado False")
            return func(*args, **kwargs)
        return wrapper
    return _dec


@dec(True)
def dummy_function(value):
    return value


print(dummy_function(42))

Un decorado también puede ser declarado como una clase, solo necesita implementar el método mágico `__call__`.

In [ ]:
class Dec:
    """Decorador sin argumentos"""
    
    def __init__(self, function):
        print("Método __init__ del decorador.")
        self.function = function

    def __call__(self, *args, **kwargs):
        print("Método __call__ del decorador.")
        return self.function(*args, **kwargs)

@Dec
def dummy_function(value):
    return value


print(dummy_function(42))

In [ ]:
class Dec:
    """Decorador con argumentos"""

    def __init__(self, param):
        # Los parámetros se pasan en el constructor, y no la función.
        self.param = param

    def __call__(self, function):
        # La función se pasa en el __call__ ahora
        def wrapped(*args, **kwargs):
            if self.param:
                print("Se ha pasado True")
            else:
                print("Se ha pasado False")
            return function(*args, **kwargs)
        return wrapped


@Dec(True)
def dummy_function(value):
    return value

print(dummy_function(42))

Además de decorar funciones, también se pueden decorar métodos de clases. Para ello, sólo hay que tener en cuenta que Python pasa de forma automática el argumento `self` a todos los métodos de clases.

In [ ]:
def dec(func):
    """Decorador reutilizable para funciones y métodos."""
    def wrapper(self=None, *args, **kwargs):
        print("Antes de ejecutar el método")
        result = func(self, *args, **kwargs)
        print("Después de ejecutar el método")
        return result
    return wrapper


class DummyClass:

    @dec
    def dummy(self, value):
        return value

    
@dec
def dummy_function(value):
    return value

obj = DummyClass()
print(obj.dummy(42))

print(dummy_function(42))

Y no solo los métodos, también se pueden crear decoradores que se pueden aplicar a una clase, permitiendo cambiar así la definición entera de la clase y de todos sus métodos.

In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        print("Antes de ejecutar el método")
        result = func(*args, **kwargs)
        print("Después de ejecutar el método")
        return result
    return wrapper

def dec_class(cls):
    """El decorador de clase recibe como primer argumento el objeto clase."""

    class NewCls:
        """Creamos una nueva clase que reemplazará a la original."""
    
        def __init__(self, *args, **kwargs):
            self.original_instance = cls(*args, **kwargs)
        
        def __getattribute__(self, name):
            """Este método se llama siempre que se accede a un método de un objeto NewCls. Esté método 
            primero intenta acceder a los atributos de NewCls, si falla, entonces accede a los de 
            self.original_instance, y si el atributo es un metodo, entonces se aplica el decorador.
            """
            try:    
                result = super().__getattribute__(name)
            except AttributeError:      
                pass
            # El else se ejecuta cuando no se lanza ninguna excepción
            else:
                return result
            result = self.original_instance.__getattribute__(name)
            if type(result) == type(self.__init__):
                return dec(result)
            else:
                return result
    return NewCls


@dec_class
class MyClass:
    
    def dummy1(self, value):
        return value
    
    def dummy2(self):
        print("dummy")


obj = MyClass()
print(obj.dummy1(42))
obj.dummy2()

## 17. Iteradores y generadores, instrucción yield.

### Iteradores

La mayoría de los contenedores y estructuras de datos que hemos visto hasta ahora pueden ser recorridas usando un bucle `for`.

```python
for element in [1, 2, 3]:
    print(element)
for element in (1, 2, 3):
    print(element)
for key in {'one':1, 'two':2}:
    print(key)
for char in "123":
    print(char)
for line in open("myfile.txt"):
    print(line, end='')
```

Cuando se ejecuta un bucle `for`, internamente se llama a la función `iter()` pasandole el contenedor. Esta función devuelve un objeto **iterador**, que es un objeto que define el método mágico `__next__()` que accede a un elemento del contendor cada vez que es llamado. Y cuando no quedan más elementos, lanza una excepción `StopIteration`.

Para hacer que tus clases funcionen de la misma forma que estos contenedores cuando se recorran con un bucle `for`, basta con definir el método `__iter__()` para que devuelva un objeto que implemente el método `__next__()`. Si esa misma clase define el método `__next__()`, basta con que `__iter__()` devuelva `self`.

In [ ]:
class Reverse:
    """Iterador para recorrer una secuenca al revés."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]


rev = Reverse('spam')
print(iter(rev))
for char in rev:
    print(char)

#### Módulo `itertools`

El módulo de `itertools` implementa una serie de constructores para crear iteradores, inspirados en lenguajes funcionales. Estandariza una serie de herramientas rápidas y eficientes en memoria que son útiles por si mismas o combinadas.

##### Iteradores infinitos

Iterador | Argumentos | Resultado | Ejemplo
---------|------------|-----------|--------
count() | start, [step] | start, start+step, start+2\*step, … | count(10) --> 10 11 12 13 14 ...
cycle() | p | p0, p1, … plast, p0, p1, … | cycle('ABCD') --> A B C D A B C D ...
repeat() | elem [,n] | elem, elem, elem, … sin fin o hasta n veces | repeat(10, 3) --> 10 10 10


##### Iteradores que terminan en la secuencia de entrada más corta

Iterador | Argumentos | Resultado | Ejemplo
---------|------------|-----------|--------
accumulate() | p [,func]|p0, p0+p1, p0+p1+p2, …|accumulate([1,2,3,4,5]) --> 1 3 6 10 15
chain() |p, q, …|p0, p1, … plast, q0, q1, … |chain('ABC', 'DEF') --> A B C D E F
chain.from_iterable() |iterable |p0, p1, … plast, q0, q1, … |chain.from_iterable(['ABC', 'DEF']) --> A B C D E F
compress() |data, selectors |(d[0] if s[0]), (d[1] if s[1]), … |compress('ABCDEF', [1,0,1,0,1,1]) --> A C E F
dropwhile() |pred, seq |seq[n], seq[n+1], emezando cuando pred falle |dropwhile(lambda x: x<5, [1,4,6,4,1]) --> 6 4 1
filterfalse() |pred, seq |elementos de seq donde pred(elem) es False | filterfalse(lambda x: x%2, range(10)) --> 0 2 4 6 8
groupby() |iterable[, key] |sub-iteradores agrupados por valor de key(v) |
islice() |seq, [start,] stop [, step] |elementos de seq[start:stop:step] |islice('ABCDEFG', 2, None) --> C D E F G
starmap() |func, seq |func(*seq[0]), func(*seq[1]), … |starmap(pow, [(2,5), (3,2), (10,3)]) --> 32 9 1000
takewhile() | pred, seq |seq[0], seq[1], hasta que pred falla |takewhile(lambda x: x<5, [1,4,6,4,1]) --> 1 4
tee() |it, n |it1, it2, … itn divide un iterador en n | 
zip_longest() |p, q, … |(p[0], q[0]), (p[1], q[1]), … |zip_longest('ABCD', 'xy', fillvalue='-') --> Ax By C- D-



##### Iteradores de combinatoria

Iterador | Argumentos | Resultado | Ejemplo
---------|------------|-----------|--------
product() |p, q, … [repeat=1] | producto cartesianso, equivalente a un bucle for anidado | product('ABCD', repeat=2) --> AA AB AC AD BA BB BC BD CA CB CC CD DA DB DC DD
permutations() |p[, r] |tuplas de longitud r, con todos los ordenes posibles, sin elementos repetidos | permutations('ABCD', 2) --> AB AC AD BA BC BD CA CB CD DA DB DC
combinations() |p, r | tuplas de longitud r, ordenadas, sin elementos repetidos | combinations('ABCD', 2) --> AB AC AD BC BD CD
combinations_with_replacement()|p, r|tuplas de longitud r, ordenadas, con elementos repetidos | combinations_with_replacement('ABCD', 2) --> AA AB AC AD BB BC BD CC CD DD



### Generadores

Los generadores son herramientas simples y sencillas para crear iteradores. Un generador se escribe igual que cualquier función, sólo que se usa la instrucción `yield` para devolver los datos, de forma que un **generador produce una secuencia de datos en vez de un valor único**.

In [ ]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1


for i in countdown(10):
    print(i)

Cuando se llama a una función generador, en realidad no se está ejecutando el código, si no que está creando un objeto generador. No se ejecuta hasta que se llama a la función `next()` con el objeto generador.

In [ ]:
def countdown(n):
    print("Cuenta atrás desde:", n)
    while n > 0:
        yield n
        n -= 1

x = countdown(10)
print(x)
next(x)

## 18. Collections, módulo de collections.

El módulo `collections` implementa contenedores especializados para ser usados como alternativas a los contenedores multipropósito que ofrece Python, como `dict`, `list`, `set` y `tuple` que hemos visto anteriormente.


### Objetos `Counter`

Un objeto `Counter` es una subclase de `dict` para contar objetos. Es una colección no ordenada donde los elementos son almacenados como claves de diccionario y sus cuentas como los valores de esas claves.




In [ ]:
import collections

c = collections.Counter()
c = collections.Counter("gandalf")
c = collections.Counter({'red': 4, 'blue': 2})
c = collections.Counter(cats=4, dogs=8)

Se puede acceder a la cuenta como si se tratara de un diccionario, pero en vez de dar una excepción cuando no existe un elemento, devuelve 0.

In [ ]:
print("Gatos:", c['cats'])
print("Hamsters:", c['hamsters'])

#### Métodos

A los métodos que tiene `dict`, `Counter` añade los siguientes.

`elements()`

Devuelve un iterador sobre los elementos, repetidos tantas veces como su cuenta. Los elementos se devuelven en un orden arbitrario, y si su cuenta es menor que 1, se ingora.

`most_common([n])`

Devuelve la lista de los *n* elementos más comunes y sus cuentas. Si se omite el parámetro *n* se muestran todos los elementos.

` subtract([iterable-or-mapping])`

Elimina de las cuentas `Counter` tantos elementos como aparezcan en el iterable pasado por parámetro.

### Objetos `deque`

Un objeto `deque` es una generalización de pilas y colas, y soportan operaciones de inserción y de eliminación eficientes y seguras desde cualquier lado de la cola.

La lista soporta operaciones similares, pero por ejemplo, el coste de insertar un elemento en el principio es de *O(n)*, mientras que en un objeto de `deque` es aproximadamente de *O(1)*.

In [ ]:
import collections

d = collections.deque('bcd')
for elem in d:
    print(elem)

#### Métodos


`append(x)`

Añade `x` al lado derecho del `deque`.

`appendleft(x)`

Añade `x` al lado izquierdo del `deque`.

`clear()`

Borra todos los elementos.

`copy()`

Crea una copia.

`count(x)`

Cuenta los elementos que sean igual a `x`.


`extend(iterable)`

Extiende por el lado derecho añadiendo los elementos del iterable pasado por argumento.

`extendleft(iterable)`

Extiende por el lado izquierdo añadiendo los elementos del iterable pasado por argumento.

`index(x[, start[, stop]])`

Devuelve la posicion del primer elemento `x` que encuentre.

`insert(i, x)`

Inserta `x` en la posición `i`.

`pop()`

Elimina y devuelve el primer lemento del lado derecho.

`popleft()`

Elimina y devuelve el primer lemento del lado izquierdo.

`remove(x)`

Borra el primer elemento que sea igual a `x`.

`reverse()`

Da la vuelta al orden de los elementos.

`rotate(n=1)`

Rota los elementos `n` pasos a la derecha. Si `n` es negativo, rota a la izquierda.



In [ ]:
d.append('e')
d.appendleft('a')
print(d)

d.rotate(1)
print(d)

d.rotate(-1)
print(d)

### Objetos `defaultdict`

Un objeto `defaultdict` es una subclase de `dict`, con la diferencia de que en el constructot se le puede pasar un objeto clase que será utilizado para inicializar el diccionario en caso de que se trate de acceder a una clave que no exista en ese momento.

In [ ]:
import collections

sample = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d = collections.defaultdict(list)

for key, value in sample:
    d[key].append(value)
    
print(d)

In [ ]:
def constant_factory(value):
    return lambda: value

d = collections.defaultdict(constant_factory('<missing>'))
d.update(name='John', action='ran')

print('{} {} to {}'.format(d['name'], d['action'], d['object']))

### Objetos `namedtuple`

El objeto `namedtuple` es una factoría que devuelve una subclase de `tuple`, con el nombre que se le indica por parámetro. La nueva subclase tiene campos que son accesibles como si fueran atributos de una clase.


In [ ]:
import collections


Point = collections.namedtuple('Point', ['x', 'y'])
p = Point(11, y=22) 
print(p[0] + p[1])

x, y = p
print(x, y)

print(p.x + p.y )

print(p)

### Objetos `OrderedDict`

Los objetos `OrderedDict` son como los diccionarios normales pero conservando el orden en el que los elementos fueron añadidos al diccionarios. Cuando se itera sobre un diccionario ordenado, los elementos se devuelven en el orden que fueron añadidos.

In [ ]:
import collections


d = {'banana': 3, 'apple': 4, 'pear': 1, 'orange': 2}

od = collections.OrderedDict(sorted(d.items(), key=lambda t: t[0]))
print(od)

od = collections.OrderedDict(sorted(d.items(), key=lambda t: t[1]))
print(od)

od = collections.OrderedDict(sorted(d.items(), key=lambda t: len(t[0])))
print(od)


## 19. Logging, el módulo de logging


El módulo de `logging` es parte de Python desde la versión 2.3, y sirve a varios propósitos:

- Registro de eventos relacionados con las operaciones de la aplicación para su diagnóstico.
- Registrar eventos para el análisis de negocio.
- Mostrar información, alternativa a `print`.


En general, `print` sólo es mejor que `logging` cuando el objetivo es mostrar información de ayuda en una aplicación de línea de comandos. Pero en general, las razones por las que `logging` es mejor que `print` son:

- El registro de log contiene información como el nombre del fichero, su ruta, la función desde donde se llama, y la línea del evento de logging.
- Los eventos de los módulos que se incluyen son automaticamente accesibles desde el logger raíz.
- El logging puede ser silenciado de forma selectiva.

In [ ]:
import logging

logging.warning('Watch out!')
logging.info('I told you so')

Hay por lo menos tres formas de configurar un logger:

- Usando un fichero INI.
    - **Pro**: Es posible actualizar la configuración durante la ejecución.
    - **Contra**: Menos control que cuando se confiugra en código.
- Usando un diccionario o un fichero con formato JSON.
    - **Pro**: Además de poder actualizar la configuración durante la ejecución, es posible cargarla desde un fichero usando el módulo `json`.
    - **Crontra**: Menos control que cuando configuras con código.
- Usando código.
    - **Pro**: Completo control de la configuración
    - **Contra**: Las modificaciones requieren modificar el código fuente

### Configuración vía fichero INI

Supongamos que tenemos el fichero `logging_config.ini` con la siguiente configuración para el logging.

```ini
[loggers]
keys=root

[handlers]
keys=stream_handler

[formatters]
keys=formatter

[logger_root]
level=DEBUG
handlers=stream_handler

[handler_stream_handler]
class=StreamHandler
level=DEBUG
formatter=formatter
args=(sys.stderr,)

[formatter_formatter]
format=%(asctime)s %(name)-12s %(levelname)-8s %(message)s
```

Podemos usar `logging.config.fileConfig()` en el código.

In [ ]:
import logging
from logging.config import fileConfig


fileConfig('logging_config.ini')
logger = logging.getLogger()
logger.debug('often makes a very good meal of %s', 'visiting tourists')

### Configuración vía diccionario

In [ ]:
import logging
from logging.config import dictConfig


logging_config = dict(
    version = 1,
    formatters = {
        'f': {'format': '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'}
    },
    handlers = {
        'h': {
                'class': 'logging.StreamHandler',
                'formatter': 'f',
                'level': logging.DEBUG
        }
    },
    root = {
        'handlers': ['h'],
        'level': logging.DEBUG,
    },
)

dictConfig(logging_config)

logger = logging.getLogger()
logger.debug('often makes a very good meal of %s', 'visiting tourists')


### Configuración en código

In [ ]:
import logging

logger = logging.getLogger()
logger.handlers = []

handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

logger.debug('often makes a very good meal of %s', 'visiting tourists')

## Ejercicios

### Decorador para medir tiempos

Python tiene muchos módulos estándar muy útiles, uno de ellos es el módulo `time`, que proporciona funciones para la gestión del tiempo.

La función más sencilla sería obtener el timestamp actual.

In [ ]:
import time

time.time()

El objetivo de este ejercicio es desarrollar un decorador que permita medir el tiempo que tarda en ejecutarse los métodos de una clase y que el resultado se pueda mostrar o por pantalla o pueda ser guardado en un fichero.

### Decorador para medir tiempos todos los métodos de clase

Usando el decorador anterior, crea un decorador que pueda ser usado para medir los tiempos de todos los métodos de una clase.

### Decorador con logger

Cambia el decorador para que use un `logger` en vez de un `print` o un fichero para mostrar la información.

### Fibonacci como un generador

El objetivo de este ejercicio es reescribir la funcíon de Fibonacci para que genere de forma infinita todos los números de esta secuencia.

### Palabras más frecuentes (2)
Cambia el código para obtener las palabras más frecuentes de La Isla del Tesoro para usar un objeto `Counter`.

### Actualización de simulador de notificaciones (1)

Cambia el código desarrollado en el ejercicio del simulador de notificaciones para que la bandeja de entrada del usuario funcione con un `OrderedDict` indexando con el código del mensaje y ordenado según la fecha de entrega.

### Actualización de simulador de notificaciones (2)

Añade al simulador de notificaciones información de logging para poder visualizar cuándo se recibe un mensaje y cuándo lo abre un usuario y cuando falla un envío.